# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Começando a entender a base

- **Podemos utilizar o ydata-profiling (antigo Pandas Profiling) para começar a entender essa base**
    - https://github.com/ydataai/ydata-profiling

In [120]:
# Importando o pandas
import pandas as pd

In [121]:
# Visualizando a base de treino
treino = pd.read_csv('train_titanic.csv')
treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [122]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

# Importando o ProfileReport do ydata-profiling
from ydata_profiling import ProfileReport



Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [123]:
# Gerando a visualização
profile = ProfileReport(treino, title='titanic_treino')

In [124]:
# Criando o relatório
profile.to_file('titanic_treino.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

- **Agora vamos analisar essa base**

In [125]:
# Verificando as informações da base
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [126]:
# Verificando o tipo dos dados
treino.dtypes.value_counts()

int64      5
object     5
float64    2
dtype: int64

In [127]:
# E os valores nulos
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

- **Agora vamos visualizar a base de teste**

In [128]:
# Visualizando a base de teste
teste = pd.read_csv('test_titanic.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [129]:
# Verificando as informações da base
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [130]:
# Analisando os valores nulos
teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

- Temos colunas que **possuem valores vazios na base de teste que não estão vazias na base de treino** (nesse caso, precisaríamos tratar essas colunas apenas na base de teste)

### Realizando os tratamentos iniciais dos dados

- **Podemos começar tratando os valores vazios**

In [131]:
# Verificando novamente valores vazios para a base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [132]:
# Retornando o shape da base
treino.shape

(891, 12)

In [133]:
# Visualizando novamente a base
treino.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [134]:
# Verificando a cardinalidade dos dados
treino.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

- Colunas com **alta cardinalidade** como nome, número do ticket e até mesmo o código da cabine não vão ajudar o modelo pois podem torná-lo **menos generalizável**. Nesse caso, vamos **eliminar essas colunas**

In [135]:
# Eliminando para a base de treino
treino= treino.drop(['Name','Ticket','Cabin'],axis=1)

In [136]:
# Também eliminando para a base de teste
teste= teste.drop(['Name','Ticket','Cabin'],axis=1)

- Agora precisamos tratar a coluna **Age**, que possui valores vazios tanto no treino quanto no teste
    - Como essa coluna possui a idade dos passageiros, podemos utilizar a **média das idades** para substituir os valores vazios

In [137]:
# Verificando a média das idades para a base de treino
treino.Age.mean()

29.69911764705882

In [138]:
# Podemos agora selecionar as colunas com idade vazia e substituir por essa média
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()

In [139]:
# E fazendo o mesmo para a base de teste
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

- Para a base de treino, ainda temos 2 valores vazios na coluna **Embarked**
    - Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela **moda**

In [140]:
# Verificando os valores na coluna Embarked
treino.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [141]:
# Selecionando a moda dessa coluna
treino.Embarked.mode()[0]

'S'

In [142]:
# Substituindo os valores vazios
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

- Para finalizar, para a base de teste ainda temos 1 valor vazio em **Fare**
    - Podemos utilizar a mesma lógica que fizemos para a coluna Age e substituir pela **moda**

In [143]:
# Verificando a média da coluna Fare para a base de teste
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

- **Verificando novamente os valores vazios**

In [144]:
# Para a base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [145]:
# Para a base de teste
teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

### Considerando apenas as colunas que não são de texto

In [146]:
# Verificando as colunas de texto na base de treino
col_treino_nr=treino.columns[treino.dtypes != 'object']
col_treino_nr

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [147]:
# Selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:,col_treino_nr]

In [148]:
# E para a base de teste
col_teste_nr=teste.columns[teste.dtypes != 'object']
col_teste_nr

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [149]:
# e os valores numéricos da base de teste
teste_nr = teste.loc[:,col_teste_nr]

### Podemos agora selecionar um modelo para classificar esses dados
- Para começar, vamos testar entre:
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [150]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [151]:
# Separando a base de treino em X e y
X= treino_nr.drop(['PassengerId','Survived'],axis=1)
Y=treino.Survived

In [152]:
# Separando em treino e validação
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size=0.33,random_state=42)

- Para a **árvore de classificação**

In [153]:
# Fazendo a importação
from sklearn import tree

In [154]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [155]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,Y_train)

In [156]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [180]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [182]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [183]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,Y_train)

In [184]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [186]:
# Importando
from sklearn.linear_model import LogisticRegression

In [188]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42)

In [189]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,Y_train)

In [190]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

### E, para finalizar, vamos avaliar esses modelos
- Para fazer essa análise, podemos usar:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [191]:
# Importando
from sklearn.metrics import accuracy_score

In [192]:
# Para a árvore
accuracy_score(Y_val,y_pred_ac)

0.6169491525423729

In [193]:
# Para o knn
accuracy_score(Y_val,y_pred_knn)

0.6542372881355932

In [194]:
# Para a regressão logística
accuracy_score(Y_val,y_pred_rl)

0.7254237288135593

- Avaliando a **matriz de confusão**

In [195]:
# Importando
from sklearn.metrics import confusion_matrix

In [204]:
# Para a árvore
confusion_matrix(Y_val,y_pred_ac)

array([[125,  50],
       [ 63,  57]], dtype=int64)

In [200]:
# Para o knn
confusion_matrix(Y_val,y_pred_knn)

array([[133,  42],
       [ 60,  60]], dtype=int64)

In [201]:
# Para a regressão logística
confusion_matrix(Y_val,y_pred_rl)

array([[156,  19],
       [ 62,  58]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [205]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792


In [206]:
# Visualizando a base de teste (apenas com valores numéricos)
teste_nr.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875


In [207]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste_nr.drop("PassengerId",axis=1)

In [208]:
# Utilizando a regressão logística na base de teste
Y_pred = clf_rl.predict(X_teste)

In [209]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived']=Y_pred

In [211]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio=teste[['PassengerId','Survived']]

In [212]:
# Exportando para um csv
base_envio.to_csv('resultadoTitanic.csv',index=False)